In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from keras import layers
import re
import matplotlib.pyplot as plt



2023-09-03 21:32:01.179504: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-03 21:32:01.221192: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-03 21:32:01.222058: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-03 21:32:01.897421: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
admission = pd.read_csv('./mimic-iv-2.2/hosp/admissions.csv.gz',nrows=1000000)

In [4]:
patients = pd.read_csv('./mimic-iv-2.2/hosp/patients.csv.gz', nrows=1000000)

In [5]:
fSubID = pd.read_csv('./mimic-iv-2.2/hosp/diagnoses_icd.csv.gz', nrows=1000000)

dfIcd = pd.read_csv('./mimic-iv-2.2/hosp/d_icd_diagnoses.csv.gz', nrows=1000000)

Since we have to convert them to numerical values, might as well do it now :D

In [6]:
admission['admittime'] = pd.to_datetime(admission['admittime'])
admission['dischtime'] = pd.to_datetime(admission['dischtime'])
admission['edregtime'] = pd.to_datetime(admission['edregtime'])
admission['edouttime'] = pd.to_datetime(admission['edouttime'])
admission['deathtime'] = pd.to_datetime(admission['deathtime'])

In [ ]:
reference_datetime = pd.Timestamp('2023-01-01')  
admission['admittime'] = (admission['admittime'] - reference_datetime).dt.total_seconds()
admission['dischtime'] = (admission['dischtime'] - reference_datetime).dt.total_seconds()
admission['edregtime'] = (admission['edregtime'] - reference_datetime).dt.total_seconds()
admission['edouttime'] = (admission['edouttime'] - reference_datetime).dt.total_seconds()
admission['deathtime'] = (admission['deathtime'] - reference_datetime).dt.total_seconds()


admission['deathtime'] = admission['deathtime'].fillna(0)




In [ ]:
admission

In [ ]:
dfIcd

fSubID

In [7]:
diagnoses = pd.merge(fSubID, dfIcd, on='icd_code')

In [ ]:
diagnoses

In [ ]:
patients

Turn this column ('anchor_year_group') into a numerical one

In [ ]:
pattern = r'(\d+) - (\d+)'
pattern1 = r'(\d+) - (\d+)'

first_age = patients['anchor_year_group'].str.extract(pattern)[0].astype(int)
second_age = patients['anchor_year_group'].str.extract(pattern1)[1].astype(int)

patients['anchor_year_group'] = second_age - first_age 






Transform 'dod' into a numerical column

In [ ]:
patients['dod'] = patients['dod'].fillna(0)


patients['dod'] = pd.to_datetime(patients['dod'])
patients['dod'] = (patients['dod'] - reference_datetime).dt.total_seconds()


In [ ]:
admission

In [8]:
patients = pd.merge(patients, admission, on='subject_id', how='left')

In [ ]:
patients

In [9]:
def check_readmission(subdf):
    subdf = subdf.sort_values(by='admittime')
    time_differences = (subdf['admittime'].iloc[1:].reset_index(drop=True) - 
      subdf['dischtime'].iloc[0:-1].reset_index(drop = True)
    )
    readmission = (time_differences.dt.days < 30).astype(int)
    readmission = np.append(readmission,0)
    return readmission

In [ ]:
def maskFor30Days(df):
    time_differences = (df['admittime'].iloc[1:].reset_index(drop=True) - 
                        df['dischtime'].iloc[0:-1].reset_index(drop=True)
    )
    readmission =(time_differences.dt.days<=30)
    readmission = np.append(readmission, False)
    return readmission

In [ ]:
def chatgptMask(df):
    time_differences = df['admittime'].shift(-1) - df['dischtime']
    readmission = time_differences <= (30 * 24 * 60 * 60)  
    readmission.iloc[-1] = False
    return df[readmission]


In [ ]:
finalDf = check_readmission(patients)

In [ ]:
finalDf

In [10]:
target = (patients.groupby('subject_id').apply(check_readmission)).explode().to_frame()
target.reset_index(inplace=True)
target.columns = ['subject_id','target']

In [ ]:
target['target'].value_counts(normalize = True)

In [11]:
hospitalitizions = pd.merge(target, diagnoses, on='subject_id')


In [ ]:
hospitalitizions = chatgptMask(hospitalitizions)

In [ ]:
hospitalitizions

In [12]:
admissions_per_patient = hospitalitizions.groupby('subject_id').size().reset_index(name='count_adm')

hospitalitizions = pd.merge(hospitalitizions, admissions_per_patient, on='subject_id')


Removing the annoying columns....

In [14]:
pattern = r'.*_y$'

annoying_columns = [col for col in hospitalitizions.columns if re.match(pattern, col)]

hospitalitizions = hospitalitizions.drop(columns=annoying_columns)

Renaming the not so annoying columns

In [17]:
pattern = r'.*_x$'

for col in patients.columns:
    if re.match(pattern, col):
        new_col_name = col[:-2] 
        patients.rename(columns={col: new_col_name}, inplace=True)


In [18]:
hospitalitizions

,subject_id,target,hadm_id,seq_num,icd_code,icd_version_x,long_title,count_adm
0,10000032,0,22595853,1,5723,9,Portal hypertension,164
1,10000032,0,22595853,2,78959,9,Other ascites,164
2,10000032,0,22841357,2,78959,9,Other ascites,164
3,10000032,0,25742920,2,78959,9,Other ascites,164
4,10000032,0,29079034,5,78959,9,Other ascites,164
...,...,...,...,...,...,...,...,...
8358896,12120444,0,24125093,9,B370,10,Candidal stomatitis,12
8358897,12120444,0,24125093,12,M353,10,Polymyalgia rheumatica,12
8358898,12120444,0,24125093,8,Q211,10,Atrial septal defect,12
8358899,12120444,0,24125093,11,G939,10,"Disorder of brain, unspecified",12


In [ ]:
hospitalitizions['long_title'].value_counts()

In [ ]:
mostAdmitted = hospitalitizions['subject_id'].value_counts().index[0]

In [ ]:
hospitalitizions[hospitalitizions['subject_id'] == mostAdmitted]

# Lets categorize columns <p>
But let's automatize it :D

In [19]:

numerical_columns = hospitalitizions.select_dtypes(exclude='object')
numerical_columns_names = numerical_columns.columns

numerical_columns_names = numerical_columns_names.to_list()

categorical_columns = hospitalitizions.drop(columns=numerical_columns_names)

categorical_columns_names = categorical_columns.columns.to_list()

In [ ]:
numerical_columns
categorical_columns

In [ ]:
categorical_columns_names

In [20]:
dictionary = {}
for col_name in categorical_columns_names:
    unique_values = hospitalitizions[col_name].unique()
    mapping = {value : i+1 for i, value in enumerate(unique_values)}
    dictionary[col_name] = mapping


for col_name, mapping in dictionary.items():
    hospitalitizions[col_name] = hospitalitizions[col_name].map(mapping)


In [ ]:
hospitalitizions['count_adm'].value_counts()


33 races?? Goodness gracious <p> Most importantly, 'Portuguese' is a race :D <p>  . <p> .  <p>   .<p> .  <p> Is 'Amadora' also a race :O

In [ ]:
hospitalitizions['race'].value_counts().count()

In [ ]:
categorical_columns['insurance'].value_counts()

In [ ]:
categorical_columns['marital_status'].value_counts()

plt.plot('insurance', 'language', data=hospitalitizions)
plt.xlabel('count_adm')
plt.ylabel('language')
plt.show()

In [ ]:
#numerical_colums = hospitalitizions.select_dtypes(exclude=['object'])
hospitalitizions.columns.to_list()

In [21]:
hospitalitizions

,subject_id,target,hadm_id,seq_num,icd_code,icd_version_x,long_title,count_adm
0,10000032,1,22595853,1,1,9,1,164
1,10000032,1,22595853,2,2,9,2,164
2,10000032,1,22841357,2,2,9,2,164
3,10000032,1,25742920,2,2,9,2,164
4,10000032,1,29079034,5,2,9,2,164
...,...,...,...,...,...,...,...,...
8358896,12120444,1,24125093,9,2582,10,2478,12
8358897,12120444,1,24125093,12,2599,10,2049,12
8358898,12120444,1,24125093,8,2602,10,2494,12
8358899,12120444,1,24125093,11,7676,10,7339,12


In [ ]:
features = hospitalitizions[['subject_id','hadm_id','seq_num','icd_code', ]]
target = hospitalitizions['count_adm','target']


target.loc[target == 1] = 0
target.loc[target > 1] = 1


In [ ]:
target.value_counts(normalize=True)

# Logistic Regression


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42)

modelLogistic = LogisticRegression()

modelLogistic.fit(X_train, y_train)

predictions = modelLogistic.predict(X_test)

matrix = confusion_matrix(y_test, predictions)





In [ ]:
accuracy = accuracy_score(y_test, predictions)
accuracy
matrix


# CNN(Convolutional Neural Network) 
- How I choose the features:
  - Read some papers that talked about the age, the time of the year they were admitted and discharged.<p>And figured the disease was also important

In [33]:
smaller_just_so_computer_can_handler['count_adm'].value_counts()

1    999855
0       145
Name: count_adm, dtype: int64

In [38]:
df = pd.read_csv('LR_1df.csv')

In [42]:
df

,subject_id,admission_type,discharge_location,marital_status,race,target,BINARY_DEATH,gender,anchor_age,count_icu,count_adm,icd_code,long_title
0,10000826,EW EMER.,HOME,SINGLE,WHITE,1,0,F,32,0.0,3,5712,Alcoholic cirrhosis of liver
1,10000883,DIRECT EMER.,HOME,SINGLE,WHITE,1,0,M,20,0.0,2,29633,"Major depressive affective disorder, recurrent..."
2,10000935,EU OBSERVATION,NaN,SINGLE,BLACK/AFRICAN AMERICAN,1,0,F,52,0.0,5,78701,Nausea with vomiting
3,10001186,SURGICAL SAME DAY ADMISSION,HOME,MARRIED,WHITE,1,0,F,46,0.0,3,99832,Disruption of external operation (surgical) wound
4,10001217,EW EMER.,HOME HEALTH CARE,MARRIED,WHITE,1,0,F,55,2.0,2,3240,Intracranial abscess
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24791,19998626,DIRECT EMER.,HOME,SINGLE,WHITE,1,0,F,35,0.0,2,56949,Other specified disorders of rectum and anus
24792,19998843,EW EMER.,DIED,SINGLE,UNKNOWN,1,0,M,45,1.0,2,80325,"Other closed skull fracture with subarachnoid,..."
24793,19999303,DIRECT EMER.,HOME,DIVORCED,WHITE,1,0,F,61,0.0,2,99889,Other specified complications of procedures no...
24794,19999379,EU OBSERVATION,NaN,MARRIED,WHITE,1,0,F,91,0.0,2,7804,Dizziness and giddiness


In [22]:
smaller_just_so_computer_can_handler = hospitalitizions.sample(n=1000000, random_state=42)


In [23]:
smaller_just_so_computer_can_handler.loc[smaller_just_so_computer_can_handler['count_adm'] == 1] = 0
smaller_just_so_computer_can_handler.loc[smaller_just_so_computer_can_handler['count_adm'] > 1] = 1

In [36]:

features = hospitalitizions.drop(columns={'icd_version_x' ,'target'})
target = hospitalitizions['target']

In [37]:




X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    #layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    #layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train_scaled, y_train, batch_size=32, epochs=10)
model.evaluate(X_test_scaled, y_test)

Epoch 1/10
195912/195912 [==============================] - 244s 1ms/step - loss: -5375999488.0000 - accuracy: 0.6210
Epoch 2/10
195912/195912 [==============================] - 249s 1ms/step - loss: -74476322816.0000 - accuracy: 0.6210
Epoch 3/10
195912/195912 [==============================] - 239s 1ms/step - loss: -313197101056.0000 - accuracy: 0.6210
Epoch 4/10
195912/195912 [==============================] - 260s 1ms/step - loss: -829893181440.0000 - accuracy: 0.6210
Epoch 5/10
195912/195912 [==============================] - 252s 1ms/step - loss: -1733787123712.0000 - accuracy: 0.6210
Epoch 6/10
195912/195912 [==============================] - 253s 1ms/step - loss: -3132496805888.0000 - accuracy: 0.6210
Epoch 7/10
195912/195912 [==============================] - 254s 1ms/step - loss: -5135948316672.0000 - accuracy: 0.6210
Epoch 8/10
195912/195912 [==============================] - 263s 1ms/step - loss: -7850800185344.0000 - accuracy: 0.6210
Epoch 9/10
195912/195912 [=============

[-18432036700160.0, 0.6203134059906006]